# Regression 02 : AirQuality

#### 머신러닝 코드 구조 <br>

![이미지](https://github.com/DA4BAM/dataset/blob/master/new_code.png?raw=true "code step1")



# [3] 새로운 알고리즘 KNN(k-Nearst Neighber) + Scaling

새로운 알고리즘을 적용하는데 알아야 할 것은 다음의 5가지 입니다.
1. 알고리즘 원리
2. 전제조건
3. 문법(함수 명 등)
4. 성능 튜닝 포인트(Hyper Parameter 튜닝)
    - 복잡도를 결정하는 요소(4번과 밀접한 관련이 있음)

In [135]:
#라이브러리들을 불러오자.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [136]:
data_path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/airquality_simple.csv'
data = pd.read_csv(data_path)

## 20.데이터 준비

### 21.변수 정리

### 22.NA 처리

### 23.Feature Engineering

* 날짜 변수 만들기  
airquality 데이터는 1973년 5~9월까지 데이터 입니다.  
month, day 데이터를 날짜 형식으로 변환해 봅시다.

(중요!)날짜변수를 만드는 이유!  
* 날짜로 부터 추가 변수를 도출해 내기 위해서 날짜 변수를 만듭니다.  
* 만약 추가변수 도출이 끝나면, 날짜변수는 제거합니다.

In [137]:
# 날짜를 yyyymmdd로 만든 후 변환해봅시다.
data['Date'] = pd.to_datetime(1973*10000 + data.Month*100 + data.Day
                              ,format='%Y%m%d')

# 요일을 추가해 봅시다. 
data['WeekDay'] = data.Date.dt.dayofweek

# working day 여부
data['WorkingDay'] = 1
data.loc[data['WeekDay'].isin([0,6]), 'WorkingDay'] = 0

# 전날 오존 농도를 추가합니다.
data['Ozone_lag1'] = data['Ozone'].shift() 

# 전날 온도, 전날 바람세기, 전날 solar.R
data['Temp_lag1'] = data['Temp'].shift() 
data['Wind_lag1'] = data['Wind'].shift() 
data['Solar.R_lag1'] = data['Solar.R'].shift() 

# 전전날 오존농도 대비 전날 오존농도 증감
data['Ozone_diff'] = data['Ozone_lag1'] -  data['Ozone'].shift(2)
data.head()

# 7일 이동평균 오존농도
data['Ozone_MA_7_lag1'] = data['Ozone'].rolling(7, min_periods=1).mean().shift()

# 3일 이동평균 온도를 추가합시다.
data['Temp_MA_3_lag1'] = data['Temp'].rolling(3, min_periods=1).mean().shift()
data.head()

,Ozone,Solar.R,Wind,Temp,Month,Day,Date,WeekDay,WorkingDay,Ozone_lag1,Temp_lag1,Wind_lag1,Solar.R_lag1,Ozone_diff,Ozone_MA_7_lag1,Temp_MA_3_lag1
0,41,190.0,7.4,67,5,1,1973-05-01,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,36,118.0,8.0,72,5,2,1973-05-02,2,1,41.0,67.0,7.4,190.0,NaN,41.000000,67.000000
2,12,149.0,12.6,74,5,3,1973-05-03,3,1,36.0,72.0,8.0,118.0,-5.0,38.500000,69.500000
3,18,313.0,11.5,62,5,4,1973-05-04,4,1,12.0,74.0,12.6,149.0,-24.0,29.666667,71.000000
4,19,NaN,14.3,56,5,5,1973-05-05,5,1,18.0,62.0,11.5,313.0,6.0,26.750000,69.333333


In [35]:
# data.fillna(method = 'bfill', inplace=True)

### 24.Dummy Variable
    * 범주형 변수를 숫자로 만드는 방법
    * pd.get_dummies, pd.concat, (Pandas Dataframe).drop
    * 불필요한 칼럼들 제거


In [138]:
# dummy variable
data_wd = pd.get_dummies(data['WeekDay'] , prefix= 'W', drop_first=1)

In [139]:
data2 = pd.concat([data, data_wd], axis=1)

* 불필요한 변수를 제거합시다.

In [140]:
# 칼럼삭제
drop_x = ['Solar.R','Wind','Temp','Month','Day','Date','WeekDay']
data2.drop(drop_x, axis = 1, inplace = True)

### 25.Data Split

데이터셋을 나눠봅시다.  
위 코드를 참조해서 직접 코드를 작성합시다. 


In [141]:
# 필요한 함수를 불러옵니다.
from sklearn.model_selection import train_test_split

In [142]:
# features와 target 분리
X = data2.drop('Ozone', axis = 1)
y = data2.iloc[:, 0] 

In [143]:
# 전체에서 train : test = 7 : 3
train_x, test_x, train_y, test_y = train_test_split( X, y, test_size = .3, random_state = 2020)

In [144]:
type(train_x)

pandas.core.frame.DataFrame

### (Again)22.NA 처리 : knn Imputation
(아래 내용은 이번 과정에서는 설명을 하지 않고 그냥 실행하고 넘어갑시다.)

In [ ]:
# 아래 코드는 data frame에 사용.
print(train_x.isnull().sum())
print(test_x.isnull().sum())


In [146]:
# 필요한 함수 불러오기
from sklearn.impute import KNNImputer

# 선언한다.
imputer = KNNImputer()

# 함수를 만든다.
imputer.fit(train_x)

# 적용한다.
train_x = imputer.transform(train_x)
test_x = imputer.transform(test_x)

In [147]:
type(train_x)

numpy.ndarray

In [148]:
# 아래코드는 넘파이 어레이에서 사용.
print(np.isnan(train_x).sum(axis=0))
print(np.isnan(test_x).sum(axis=0))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0]


### 26.Scaling features
knn 알고리즘으로 모델링 하기 위해서는 반드시 scaling이 필요!



In [149]:
# 필요한 함수 로딩
from sklearn.preprocessing import MinMaxScaler

In [150]:
# 함수 선언
scaler = MinMaxScaler()

# 함수 만들기
scaler.fit(train_x)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [151]:
# scaling 전( np.array)
print(train_x.min(axis=0).round(2))
print(train_x.max(axis=0))

[  0.     1.    56.     2.8    7.   -95.    13.43  58.67   0.     0.
   0.     0.     0.     0.  ]
[  1.         168.          97.          20.7        334.
 123.          98.71428571  95.66666667   1.           1.
   1.           1.           1.           1.        ]


In [152]:
# 함수 적용하기
train_x = scaler.transform(train_x)
test_x = scaler.transform(test_x)

In [153]:
# scaling 후 ( np.array)
print(train_x.min(axis=0))
print(train_x.max(axis=0))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


### 27.Dataframe to Numpy array

In [128]:
type(train_x)

numpy.ndarray

sk-learn의 transform 함수를 통해 변환된 결과물은 numpy array가 됩니다.  
그러므로 다시 할 필요가 없습니다. 

## 30.모델링 : knn

### 31.import

[K-Nearest Neighbors Regression](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html#sklearn.neighbors.KNeighborsRegressor)

검색요령 : sklearn <알고리즘> 회귀?분류?  
예 : knn 회귀 ==> sklearn knn regressssor  
svm 분류 ==> sklearn svm classifier



In [154]:
# 검색창에, sklearn knn regressor 로 검색합시다.
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

### 32.모델선언

k 값에 따라 성능은 달라집니다. 하지만 여기서는 default(5)로 두고 모델링 합니다.

In [155]:
model_knn = KNeighborsRegressor()

### 33.모델링(학습)

In [156]:
model_knn.fit(train_x, train_y)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')

### 34.예측

In [157]:
test_pred = model_knn.predict(test_x)

### 35.평가

In [158]:
mean_squared_error(test_y, test_pred, squared=0)

26.75906997798983

In [163]:
# 수동으로 grid search 방법 튜닝 시행

for k in range(3,21) :
    model_knn = KNeighborsRegressor(n_neighbors=k)
    model_knn.fit(train_x, train_y)
    test_pred_knn = model_knn.predict(test_x)
    print('k:{}, RMSE : {}'.format(k, mean_squared_error(test_y, test_pred_knn, squared=0)))


k:3, RMSE : 29.539051365193764
k:4, RMSE : 27.011118886085796
k:5, RMSE : 26.75906997798983
k:6, RMSE : 26.716495564832233
k:7, RMSE : 26.986267828038052
k:8, RMSE : 26.924165089392645
k:9, RMSE : 27.196951693669728
k:10, RMSE : 27.034386315917377
k:11, RMSE : 27.638720125246323
k:12, RMSE : 27.764005039716636
k:13, RMSE : 27.80564414204149
k:14, RMSE : 27.818246056583003
k:15, RMSE : 27.809677528716644
k:16, RMSE : 27.248027326882248
k:17, RMSE : 27.0340415260767
k:18, RMSE : 26.876119857081218
k:19, RMSE : 26.38371211494832
k:20, RMSE : 26.482778448414848


Linear Regression 과 knn 어떤 알고리즘으로 모델링한 결과가 더 나은가요?


In [159]:
from sklearn.linear_model import LinearRegression

model_lm = LinearRegression()
model_lm.fit(train_x, train_y)
test_pred_lm = model_lm.predict(test_x)
mean_squared_error(test_y, test_pred_lm, squared=0)


25.17172567949234